# load libraries and helper functions

In [1]:
from HCP_helpers import *

from pathlib import Path
import os.path as op
scrPath = Path(os.getcwd())
rootPath = scrPath.parent.parent
print(rootPath)
dfPath = op.join(rootPath, 'unrestricted_11_4_2020_2_19_35.csv')


/data/home/attpit01/SNregression_yeo


/data/home/attpit01/SNregression_yeo/scripts/HCP_helper/HCP_helpers.py:81: UserWarning: 

 | Using Nistats with Nilearn versions >= 0.7.0 is redundant and potentially conflicting.
 | Nilearn versions 0.7.0 and up offer all the functionality of Nistats as well the latest features and fixes.
 | We strongly recommend uninstalling Nistats and using Nilearn's stats & reporting modules.

  import nistats


# subjects selection

In [2]:

df = pd.read_csv(dfPath)
df
# keep variables of interest
df = df[['Subject','Release','Gender','Age','fMRI_3T_ReconVrs',
        'FS_BrainSeg_Vol','MMSE_Score','3T_RS-fMRI_PctCompl',
        'PMAT_Compl','NEO-FFI_Compl','MMSE_Compl',
        'Non-TB_Compl','VisProc_Compl','DelDisc_Compl',
        'SCPT_Compl','IWRD_Compl','VSPLOT_Compl',
        'CardSort_Unadj','Flanker_Unadj','ListSort_Unadj',
        'PicSeq_Unadj','PicVocab_Unadj','ProcSpeed_Unadj',
        'ReadEng_Unadj','IWRD_TOT','PMAT24_A_CR','VSPLOT_TC']]

df['Gender'].replace(['F', 'M'],[1, 2],inplace=True)
df['fMRI_3T_ReconVrs'].replace(['r177','r177 r227','r227'],[1,2,3],inplace=True)

keepSub = (df['Release'] == 'Q2') | (df['Release'] == 'Q1')

# keepSub = ((df['Release'] == 'Q1') | (df['Release'] == 'Q2') | (df['Release'] == 'Q3') 
#            | (df['Release'] == 'S500') | (df['Release'] == 'S900') | (df['Release'] == 'S1200') 
#            | (df['Release'] == 'MEG2'))

print('Selected {} subjects for release {}'.format(np.sum(keepSub),['Q1 & Q2']))

# select subjects that have completed all neuropsych
keepSub = keepSub & (
    (df['PMAT_Compl']==True) &
    (df['NEO-FFI_Compl']==True) &
    (df['MMSE_Compl']==True) &
    (df['Non-TB_Compl']==True) &
    (df['VisProc_Compl']==True) &
    (df['SCPT_Compl']==True) &
    (df['IWRD_Compl']==True) &
    (df['VSPLOT_Compl']==True)
    )
print('Selected {} subjects with complete neuropsych data'.format(np.sum(keepSub)))

# FURTHER EXCLUSIONARY CRITERIA: MISSING VALUES
keepSub    = np.logical_and(keepSub,np.logical_not(np.isnan(df['CardSort_Unadj'])))
keepSub    = np.logical_and(keepSub,np.logical_not(np.isnan(df['VSPLOT_TC'])))
keepSub    = np.logical_and(keepSub,np.logical_not(np.isnan(df['PicSeq_Unadj'])))
print('Kept {} subjects after removing missing values'.format(np.sum(keepSub)))

# COGNITIVE COMPROMISE --> MMSE <26 excluded
keepSub    = np.logical_and(keepSub,df['MMSE_Score']>=26)

# FURTHER PRUNE SUBJECTS FOR MRI ANALYSIS
# Exclusion of subjects who did not complete all RS-fMRI
keepSub = np.logical_and(keepSub,df['3T_RS-fMRI_PctCompl']==100)
print('Kept {} subjects with complete rfMRI datasets'.format(np.sum(keepSub)))

# Exclusion of high-motion subjects
# exclude subjects with >0.14 frame-to-frame head motion estimate averged across both rest runs (arbitrary threshold as in Finn et al 2015)
fmriRuns = ['rfMRI_REST1_LR', 'rfMRI_REST1_RL']
RelRMSMean = np.zeros([len(df['Subject']), len(fmriRuns)],dtype=np.float32)
FDsum      = np.zeros([len(df['Subject']), len(fmriRuns)],dtype=np.float32)
iSub=0
excluded = list()
RMSPath = op.join(rootPath,'RMS')
for subject in df['Subject']:
    if not keepSub[iSub]:
        iSub=iSub+1
        continue
    config.subject=str(subject)
    # RelRMSMean
    i=0
    for config.fmriRun in fmriRuns:

        RelRMSMeanFile = op.join(RMSPath,config.subject,config.fmriRun,'Movement_RelativeRMS_mean.txt')
        if op.isfile(RelRMSMeanFile):
            with open(RelRMSMeanFile,'r') as tmp:
                RelRMSMean[iSub,i] = float(tmp.read())
        else:
            keepSub[iSub]=False
            excluded.append(config.subject)
            break
        i=i+1
    if i==len(fmriRuns): # all RelRMSMeanFile exist
        if np.any(RelRMSMean[iSub,:] > 0.15):
            keepSub[iSub]=False
            excluded.append(config.subject)
        else:
            keepSub[iSub]=True
    # total framewise displacement
    i=0
    for config.fmriRun in fmriRuns:
        FDsumFile = op.join(RMSPath,config.subject,config.fmriRun, 'FD_sum.txt')
        if not op.isfile(FDsumFile):
            motionFile = op.join(RMSPath,config.subject,config.fmriRun,'Movement_Regressors_dt.txt')
            if op.isfile(motionFile):
                dmotpars = np.abs(np.genfromtxt(motionFile)[:,6:]) #derivatives
                headradius=50 #50mm as in Powers et al. 2012
                disp=dmotpars.copy()
                disp[:,3:]=np.pi*headradius*2*(disp[:,3:]/360)
                thisFDsum=np.sum(np.sum(disp,1),0)
                with open(FDsumFile,'w') as tmp:
                    tmp.write(str(thisFDsum))
            else:
                break
        with open(FDsumFile,'r') as tmp:
            FDsum[iSub,i] = float(tmp.read())
        i=i+1
    iSub=iSub+1

# add RelRMSMean and FDsum to the dataframe
df['RelRMSMean_REST1'] = np.mean(RelRMSMean[:,0:2],axis=1)
df['FDsum_REST1']      = np.mean(FDsum[:,0:2],axis=1)

print('Kept {} subjects with motion <0.15mm'.format(np.sum(keepSub)))


# PRUNE df 
df        = df[keepSub]
# reindex
df.index  = range(df.shape[0])


Selected 140 subjects for release ['Q1 & Q2']
Selected 138 subjects with complete neuropsych data
Kept 138 subjects after removing missing values
Kept 128 subjects with complete rfMRI datasets
Kept 119 subjects with motion <0.15mm


## Exploratory factor analysis

In [15]:
cogScores = ['PicVocab_Unadj',              # Vocabulary, Language, Crystallized, Global
             'ReadEng_Unadj',               # Reading, Language, Crystallized, Global
             'PicSeq_Unadj',                # Episodic memory, Fluid, Global
             'Flanker_Unadj',               # Executive, Fluid, Global
             'CardSort_Unadj',              # Executive, Fluid, Global
             'ProcSpeed_Unadj',             # Speed, Executive, Fluid, Global
             'PMAT24_A_CR',                 # non-verbal reasoning: Number of Correct Responses, Median Reaction Time for Correct Responses 
             'VSPLOT_TC',                   # Spatial ability: Total Number Correct, Median Reaction Time Divided by Expected Number of Clicks for Correct 
             'IWRD_TOT',                    # Verbal memory
             'ListSort_Unadj',              # Working memory, Executive, Fluid, Global
        ]
alpha = 1e-3
for score in cogScores:
    k2, p = stats.normaltest(df[score])
    print("{} normality test: p = {:g}".format(score,p))
cogdf      = df[cogScores].copy()

# standardize scores
standardize = lambda x: (x-x.mean()) / x.std() #* 15. + 100.
cogdf = cogdf.pipe(standardize)
cogdf.to_csv(op.join(rootPath, 'cogdf.csv'))

PicVocab_Unadj normality test: p = 0.532616
ReadEng_Unadj normality test: p = 0.215002
PicSeq_Unadj normality test: p = 0.330511
Flanker_Unadj normality test: p = 0.0481468
CardSort_Unadj normality test: p = 0.0168717
ProcSpeed_Unadj normality test: p = 4.40422e-06
PMAT24_A_CR normality test: p = 0.00347337
VSPLOT_TC normality test: p = 0.641865
IWRD_TOT normality test: p = 4.32077e-06
ListSort_Unadj normality test: p = 0.105113


## Code below should be run in the case of 'computeG' rscript has been performed

In [16]:
b4Scores = pd.read_csv(op.join(rootPath, 'b4Scores_EFA.csv')) # bi-factor score from EFA
biScores = pd.read_csv(op.join(rootPath, 'biScores_CFA.csv')) # bi-factor score from CFA
df['G']      = cogdf['CardSort_Unadj'] + cogdf['Flanker_Unadj'] + cogdf['ProcSpeed_Unadj'] + cogdf['PicVocab_Unadj'] + (
            cogdf['ReadEng_Unadj'] + cogdf['PMAT24_A_CR'] + cogdf['VSPLOT_TC'] + cogdf['IWRD_TOT'] + cogdf['PicSeq_Unadj'] + cogdf['ListSort_Unadj'])
df['g_efa']  = b4Scores.iloc[:,0]
df['g_cfa']  = biScores.iloc[:,0]
print(df[['G','g_efa','g_cfa','PMAT24_A_CR']].corr())

                    G     g_efa     g_cfa  PMAT24_A_CR
G            1.000000  0.911461  0.911439     0.638255
g_efa        0.911461  1.000000  0.977574     0.740371
g_cfa        0.911439  0.977574  1.000000     0.708680
PMAT24_A_CR  0.638255  0.740371  0.708680     1.000000


make sure that the selected subjects all have complete MRI runs and performed cognitive tests

In [3]:
subj_MRI = np.genfromtxt(op.join(rootPath, 'subjectsID_SAVE.txt'), dtype=int)
for sub in df['Subject']:
    if not sub in subj_MRI:
        print(sub)

In [18]:
scoreList  = ['g_efa','PMAT24_A_CR',
          'Gender','FS_BrainSeg_Vol',
          'FDsum_REST1']
thisdf = df[scoreList].copy()
corr   = thisdf.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
sns.set(style="white")
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(10,10))
# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, annot=True, fmt=".2f", mask=mask, cmap=cmap, vmax=.8, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.savefig(op.join(rootPath,"confounds.svg"), format='svg')

## Confound removal assessment
Use multiple regression to remove effects of confounds and assess whether there is any correlation left.

In [19]:
decon = 'decon'

if decon == 'megatrawlDecon':
    confounds=['gender','age','age^2','gender*age','gender*age^2','brainsize','motion','recon']
elif decon == 'megatrawlDecon+IQ':
    confounds=['gender','age','age^2','gender*age','gender*age^2','brainsize','motion','recon','PMAT24_A_CR']
elif decon == 'decon':
    # confounds=['gender','age','brainsize','motion','recon']
    confounds=['gender','brainsize','motion']
elif decon == 'decon+IQ':
    confounds=['gender','age','brainsize','motion','recon','PMAT24_A_CR']
elif decon == 'noDecon':
    confounds=[]

# allConfounds=['gender','age','age^2','gender*age','gender*age^2','brainsize','motion','recon','PMAT24_A_CR']
allConfounds=['gender','brainsize','motion','PMAT24_A_CR']
# make a matrix with all confounds
conMatAll = None
for confound in allConfounds:
    if confound == 'gender':
        conVec = df['Gender']
    elif confound == 'age':
        conVec = df['Age_in_Yrs']
    elif confound == 'age^2':
        conVec = np.square(df['Age_in_Yrs'])
    elif confound == 'gender*age':
        conVec = np.multiply(df['Gender'],df['Age_in_Yrs'])
    elif confound == 'gender*age^2':
        conVec = np.multiply(df['Gender'],np.square(df['Age_in_Yrs']))
    elif confound == 'brainsize':
        conVec = df['FS_BrainSeg_Vol']
    elif confound == 'motion':
        conVec = df['FDsum_REST1']
    elif confound == 'recon':
        conVec = df['fMRI_3T_ReconVrs']
    elif confound == 'PMAT24_A_CR':
        conVec = df['PMAT24_A_CR']
    # add to conMat
    if conMatAll is None:
        conMatAll = np.array(np.ravel(conVec))
    else:
        conMatAll = np.vstack((conMatAll,conVec))
conMatAll = conMatAll.T

# make a matrix with just the confounds used
conMat = None
for confound in confounds:
    if confound == 'gender':
        conVec = df['Gender']
    elif confound == 'age':
        conVec = df['Age_in_Yrs']
    elif confound == 'age^2':
        conVec = np.square(df['Age_in_Yrs'])
    elif confound == 'gender*age':
        conVec = np.multiply(df['Gender'],df['Age_in_Yrs'])
    elif confound == 'gender*age^2':
        conVec = np.multiply(df['Gender'],np.square(df['Age_in_Yrs']))
    elif confound == 'brainsize':
        conVec = df['FS_BrainSeg_Vol']
    elif confound == 'motion':
        conVec = df['FDsum_REST1']
    elif confound == 'recon':
        conVec = df['fMRI_3T_ReconVrs']
    elif confound == 'PMAT24_A_CR':
        conVec = df['PMAT24_A_CR']
    # add to conMat
    if conMat is None:
        conMat = np.array(np.ravel(conVec))
    else:
        conMat = np.vstack((conMat,conVec))
conMat = conMat.T

# check correlations with all confounds
print(allConfounds)
score = 'g_efa'
# correlations before
corrBef = []
for i in range(len(allConfounds)):
    corrBef.append(stats.pearsonr(conMatAll[:,i].T,np.ravel(df[score]))[0])
print(', '.join('{:03f}'.format(k) for k in corrBef))
# regress out confounds
regr        = linear_model.LinearRegression()
regr.fit(conMat, np.ravel(df[score]))
fittedvalues = regr.predict(conMat)
deconScore   = np.ravel(df[score]) - np.ravel(fittedvalues)
# correlations after
corrAft = []
for i in range(len(allConfounds)):
    corrAft.append(stats.pearsonr(conMatAll[:,i].T,deconScore)[0])
print(', '.join('{:03f}'.format(k) for k in corrAft))
df['g_efa_decon'] = deconScore

['gender', 'brainsize', 'motion', 'PMAT24_A_CR']
0.203744, 0.233584, -0.099561, 0.740371
-0.000000, -0.000000, -0.000000, 0.712808


In [3]:
subj_MRI = np.genfromtxt(op.join(rootPath, 'subjectsID_SAVE.txt'), dtype=int)
fc_keep = []
for sub in subj_MRI:
    if not sub in np.array(df['Subject']):
        print(sub)
    fc_keep.append(sub in np.array(df['Subject']))
df_fc = pd.read_csv(op.join(rootPath, 'fc.csv'),header=None)
df_fc = df_fc.iloc[fc_keep,:]
df_fc.to_csv(op.join(rootPath, 'fc_z_sel.csv'), index=False, header=False)

114924
119833
123117
140420
150423
151223
156637
191437
250427
677968
788876
992774
